### TODO:

1. store in feast
2. modify dataset with the following rules
    - create "standard yearly salary"
    - use the salary and pay_period:
        + yearly -> use the mid-point
        + hourly -> 40 hrs per week * number of weeks in a year
        + monthly -> * 12 months
        + bi-weekly -> * 2 * 12 months
        + else -> consider without salary attribute (use for testing phase - predict)
4. create a dataset with the following requirements
    - only all which have description, min, max salary value

In [1]:
import pandas as pd

In [6]:
df = pd.read_csv("./dataset/linkedin-job-posting/postings.csv")

In [7]:
df.dtypes, df.shape

(job_id                          int64
 company_name                   object
 title                          object
 description                    object
 max_salary                    float64
 pay_period                     object
 location                       object
 company_id                    float64
 views                         float64
 med_salary                    float64
 min_salary                    float64
 formatted_work_type            object
 applies                       float64
 original_listed_time          float64
 remote_allowed                float64
 job_posting_url                object
 application_url                object
 application_type               object
 expiry                        float64
 closed_time                   float64
 formatted_experience_level     object
 skills_desc                    object
 listed_time                   float64
 posting_domain                 object
 sponsored                       int64
 work_type               

In [8]:
df.isna().sum()

job_id                             0
company_name                    1719
title                              0
description                        7
max_salary                     94056
pay_period                     87776
location                           0
company_id                      1717
views                           1689
med_salary                    117569
min_salary                     94056
formatted_work_type                0
applies                       100529
original_listed_time               0
remote_allowed                108603
job_posting_url                    0
application_url                36665
application_type                   0
expiry                             0
closed_time                   122776
formatted_experience_level     29409
skills_desc                   121410
listed_time                        0
posting_domain                 39968
sponsored                          0
work_type                          0
currency                       87776
c

- job_id
- title
- description
- max_salary
- pay_period
- med_salary
- min_salary
- compensation_type

In [16]:
mask = (
    df["title"].notnull() & 
    df["description"].notnull() & 
    df["max_salary"].notnull() & 
    df["pay_period"].notnull() &
    df["min_salary"].notnull()
)

In [17]:
df_train = df[mask].copy()

In [18]:
df_train.dtypes, df_train.shape

(job_id                          int64
 company_name                   object
 title                          object
 description                    object
 max_salary                    float64
 pay_period                     object
 location                       object
 company_id                    float64
 views                         float64
 med_salary                    float64
 min_salary                    float64
 formatted_work_type            object
 applies                       float64
 original_listed_time          float64
 remote_allowed                float64
 job_posting_url                object
 application_url                object
 application_type               object
 expiry                        float64
 closed_time                   float64
 formatted_experience_level     object
 skills_desc                    object
 listed_time                   float64
 posting_domain                 object
 sponsored                       int64
 work_type               

In [19]:
df_predict = df[~mask].copy()

In [20]:
df_predict.dtypes, df_predict.shape

(job_id                          int64
 company_name                   object
 title                          object
 description                    object
 max_salary                    float64
 pay_period                     object
 location                       object
 company_id                    float64
 views                         float64
 med_salary                    float64
 min_salary                    float64
 formatted_work_type            object
 applies                       float64
 original_listed_time          float64
 remote_allowed                float64
 job_posting_url                object
 application_url                object
 application_type               object
 expiry                        float64
 closed_time                   float64
 formatted_experience_level     object
 skills_desc                    object
 listed_time                   float64
 posting_domain                 object
 sponsored                       int64
 work_type               

In [21]:
assert (df_train.shape[0] + df_predict.shape[0]) == df.shape[0], "not correct data size, possibly data are missed"